Зарегистрируйтесь в Yandex.Cloud.  
Создайте новый проект в Yandex.Cloud.  
В разделе "API и сервисы" подключите сервис "Yandex SpeechKit".  
Создайте API-ключ для использования Yandex SpeechKit.  

In [ ]:
pip install requests pydub

In [ ]:
import requests
from pydub import AudioSegment
import base64
import json

In [ ]:
def get_amocrm_token(client_id, client_secret, redirect_uri, code):
    url = 'https://your_amocrm_domain.amocrm.ru/oauth2/access_token'
    data = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'authorization_code',
        'code': code,
        'redirect_uri': redirect_uri 
    }
    response = requests.post(url, json=data)
    return response.json()


Функция для получения токена доступа AMOcrm

In [ ]:
def speech_to_text(audio_file_path, api_key):
    with open(audio_file_path, 'rb') as f:
        audio_content = f.read()
    base64_audio = base64.b64encode(audio_content).decode('utf-8')

    headers = {
        'Authorization': f'Api-Key {api_key}', 
        'Content-Type': 'application/json'
    }

    data = {
        'config': {
            'encoding': 'LINEAR16',
            'sampleRateHertz': 16000,
            'languageCode': 'ru-RU'
        },
        'audio': {
            'content': base64_audio
        }
    }

    response = requests.post(
        'https://stt.api.cloud.yandex.net/speech/v1/stt:recognize',
        headers=headers,
        json=data
    )
    return response.json()


Функция для преобразования речи в текст с помощью Yandex SpeechKit

In [ ]:
def create_amocrm_deal(token, deal_data):
    url = 'https://your_amocrm_domain.amocrm.ru/api/v4/leads'
    headers = {
        'Authorization': f'Bearer {token}'
    }
    response = requests.post(url, headers=headers, json=deal_data)
    return response.json()


Функция для создания сделки в AMOcrm

In [ ]:
if __name__ == "__main__":
    client_id = 'your_amocrm_client_id' # в этой функции нужно вставить свои токены
    client_secret = 'your_amocrm_client_secret'
    redirect_uri = 'your_redirect_uri'
    code = 'authorization_code_received_from_amocrm'
    
    amocrm_token = get_amocrm_token(client_id, client_secret, redirect_uri, code)['access_token']
    
    yandex_api_key = 'your_yandex_speechkit_api_key' # здесь нужно вставить свой апи от яндекс клауд
    audio_file_path = 'path_to_your_audio_file.wav'
    
    # Конвертация аудио в текст
    transcription = speech_to_text(audio_file_path, yandex_api_key)
    if 'result' in transcription:
        text = transcription['result']
        print(f'Transcription: {text}')
        
        # Создание сделки в AMOcrm
        deal_data = {
            "name": "New Deal from Speech",
            "price": 1000,
            "custom_fields_values": [
                {
                    "field_id": your_custom_field_id,
                    "values": [
                        {
                            "value": text
                        }
                    ]
                }
            ]
        }
        deal_response = create_amocrm_deal(amocrm_token, deal_data) 
        print(f'Deal created: {deal_response}')
    else:
        print('Error in transcription:', transcription)


Основной скрипт для интеграции

**В коде нужно заменить your_amocrm_domain, your_amocrm_client_id, your_amocrm_client_secret, your_redirect_uri, your_yandex_speechkit_api_key, your_custom_field_id на соответствующие ваши значения.**